In [1]:
import numpy as np
import scipy.sparse as sp
import scipy.sparse.linalg as linsolve

In [2]:
N = 100
L = 1.
dx = 1./(N - 1)
x = np.arange(0, L+.5*dx, dx)

In [3]:
A = sp.spdiags([-np.ones(N), 2*np.ones(N), -np.ones(N)],[-1, 0, 1], N, N) # 1D poisson matrix

In [4]:
A = A.tocsr()

In [5]:
A[0, 0]=1;A[0, 1]=0;
A[-1, -2]=0;A[-1, -1]=1;

In [6]:
A.todense()

matrix([[ 1.,  0.,  0., ...,  0.,  0.,  0.],
        [-1.,  2., -1., ...,  0.,  0.,  0.],
        [ 0., -1.,  2., ...,  0.,  0.,  0.],
        ..., 
        [ 0.,  0.,  0., ...,  2., -1.,  0.],
        [ 0.,  0.,  0., ..., -1.,  2., -1.],
        [ 0.,  0.,  0., ...,  0.,  0.,  1.]])

In [7]:
b = np.ones(N)*dx**2

In [8]:
b[0] = 0; b[-1] = 0

In [9]:
LU = linsolve.factorized(A.tocsc())

In [10]:
u = LU(b)

In [18]:
from bqplot import *
x_sc = LinearScale()
y_sc = LinearScale()

ax_x = Axis(label='x', scale=x_sc, grid_lines='solid')
ax_y = Axis(label='solution', scale=y_sc, orientation='vertical', grid_lines='solid')

In [12]:
line = Lines(x=x, y=u, scales={'x': x_sc, 'y': y_sc})

In [13]:
fig = Figure(axes=[ax_x, ax_y], marks=[line], animation_duration=1000)
fig

Figure(animation_duration=1000, axes=[Axis(label='x', scale=LinearScale(), side='bottom'), Axis(label='solution', orientation='vertical', scale=LinearScale(), side='left')], fig_margin={'top': 60, 'bottom': 60, 'left': 60, 'right': 60}, layout=Layout(min_width='125px'), marks=[Lines(colors=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf'], interactions={'hover': 'tooltip'}, scales={'x': LinearScale(), 'y': LinearScale()}, scales_metadata={'x': {'orientation': 'horizontal', 'dimension': 'x'}, 'y': {'orientation': 'vertical', 'dimension': 'y'}, 'color': {'dimension': 'color'}}, tooltip_style={'opacity': 0.9}, x=array([ 0.        ,  0.01010101,  0.02020202,  0.03030303,  0.04040404,
        0.05050505,  0.06060606,  0.07070707,  0.08080808,  0.09090909,
        0.1010101 ,  0.11111111,  0.12121212,  0.13131313,  0.14141414,
        0.15151515,  0.16161616,  0.17171717,  0.18181818,  0.19191919,
        0.2020202 ,  0.21212121,  0.22222222,  0.23232323,  0.24242424,
        0.25252525,  0.26262626,  0.27272727,  0.28282828,  0.29292929,
        0.3030303 ,  0.31313131,  0.32323232,  0.33333333,  0.34343434,
        0.35353535,  0.36363636,  0.37373737,  0.38383838,  0.39393939,
        0.4040404 ,  0.41414141,  0.42424242,  0.43434343,  0.44444444,
        0.45454545,  0.46464646,  0.47474747,  0.48484848,  0.49494949,
        0.50505051,  0.51515152,  0.52525253,  0.53535354,  0.54545455,
        0.55555556,  0.56565657,  0.57575758,  0.58585859,  0.5959596 ,
        0.60606061,  0.61616162,  0.62626263,  0.63636364,  0.64646465,
        0.65656566,  0.66666667,  0.67676768,  0.68686869,  0.6969697 ,
        0.70707071,  0.71717172,  0.72727273,  0.73737374,  0.74747475,
        0.75757576,  0.76767677,  0.77777778,  0.78787879,  0.7979798 ,
        0.80808081,  0.81818182,  0.82828283,  0.83838384,  0.84848485,
        0.85858586,  0.86868687,  0.87878788,  0.88888889,  0.8989899 ,
        0.90909091,  0.91919192,  0.92929293,  0.93939394,  0.94949495,
        0.95959596,  0.96969697,  0.97979798,  0.98989899,  1.        ]), y=array([ 0.        ,  0.00499949,  0.00989695,  0.01469238,  0.01938578,
        0.02397715,  0.02846648,  0.03285379,  0.03713907,  0.04132231,
        0.04540353,  0.04938272,  0.05325987,  0.057035  ,  0.06070809,
        0.06427916,  0.06774819,  0.07111519,  0.07438017,  0.07754311,
        0.08060402,  0.0835629 ,  0.08641975,  0.08917457,  0.09182736,
        0.09437812,  0.09682685,  0.09917355,  0.10141822,  0.10356086,
        0.10560147,  0.10754005,  0.10937659,  0.11111111,  0.1127436 ,
        0.11427405,  0.11570248,  0.11702887,  0.11825324,  0.11937557,
        0.12039588,  0.12131415,  0.12213039,  0.12284461,  0.12345679,
        0.12396694,  0.12437506,  0.12468115,  0.12488522,  0.12498725,
        0.12498725,  0.12488522,  0.12468115,  0.12437506,  0.12396694,
        0.12345679,  0.12284461,  0.12213039,  0.12131415,  0.12039588,
        0.11937557,  0.11825324,  0.11702887,  0.11570248,  0.11427405,
        0.1127436 ,  0.11111111,  0.10937659,  0.10754005,  0.10560147,
        0.10356086,  0.10141822,  0.09917355,  0.09682685,  0.09437812,
        0.09182736,  0.08917457,  0.08641975,  0.0835629 ,  0.08060402,
        0.07754311,  0.07438017,  0.07111519,  0.06774819,  0.06427916,
        0.06070809,  0.057035  ,  0.05325987,  0.04938272,  0.04540353,
        0.04132231,  0.03713907,  0.03285379,  0.02846648,  0.02397715,
        0.01938578,  0.01469238,  0.00989695,  0.00499949,  0.        ]))], scale_x=LinearScale(allow_padding=False, max=1.0, min=0.0), scale_y=LinearScale(allow_padding=False, max=1.0, min=0.0))

In [1]:
import ipyparallel as ipp
c = ipp.Client(profile='mpi')

In [2]:
view = c[:]
view.activate()
view.block = True

In [149]:
%%px
import mpi4py.MPI as mpi
import numpy as np
import scipy.sparse as sp
import scipy.sparse.linalg as linsolve

size = mpi.COMM_WORLD.size
rank = mpi.COMM_WORLD.rank

N = 100
L = 1.
dx = 1./(N - 1)

In [150]:
%%px
Nl = (N-1)//size + 1
if mpi.COMM_WORLD.rank == size-1:
    Nl += (N-1)%size

In [151]:
%%px
Narray = mpi.COMM_WORLD.allgather(Nl-1)

tmp = np.cumsum(Narray)
tmp = np.insert(tmp, 0, 0)

In [152]:
%px tmp

Out[0:157]: array([ 0, 24, 48, 72, 99])

Out[1:157]: array([ 0, 24, 48, 72, 99])

Out[2:157]: array([ 0, 24, 48, 72, 99])

Out[3:157]: array([ 0, 24, 48, 72, 99])

In [153]:
%%px
beg = tmp[rank]*dx
end = tmp[rank+1]*dx

overlap = 4

if rank > 0:
    beg -= dx*overlap
if rank < size - 1:
    end += dx*overlap

In [154]:
%px xl = np.arange(beg, end + .5*dx, dx)

In [155]:
%px n = xl.size

In [156]:
%%px
A = sp.spdiags([-np.ones(n), 2*np.ones(n), -np.ones(n)],[-1, 0, 1], n, n) # 1D poisson matrix
A = A.tocsr()

A[0, 0]=1;A[0, 1]=0;
A[-1, -2]=0;A[-1, -1]=1;

b = np.ones(n)*dx**2
b[0] = 0; b[-1]=0

In [157]:
%px LU = linsolve.factorized(A.tocsc())

In [158]:
%px u = LU(b)

In [159]:
%%px
def update(nbite=100):
    U = []
    U.append(mpi.COMM_WORLD.allgather(u))
    X = mpi.COMM_WORLD.allgather(xl)
    for k in range(nbite):
        if rank == 0:
            mpi.COMM_WORLD.send(u[-1-2*overlap], rank + 1, rank)
            b[-1] = mpi.COMM_WORLD.recv(None, rank + 1, rank + 1)
        elif rank == size - 1:
            mpi.COMM_WORLD.send(u[2*overlap], rank - 1, rank)
            b[0] = mpi.COMM_WORLD.recv(None, rank - 1, rank - 1)
        else:
            mpi.COMM_WORLD.send(u[-1-2*overlap], rank + 1, rank)
            b[-1] = mpi.COMM_WORLD.recv(None, rank + 1, rank + 1)
            mpi.COMM_WORLD.send(u[2*overlap], rank - 1, rank)
            b[0] = mpi.COMM_WORLD.recv(None, rank - 1, rank - 1)
        u[:] = LU(b)
        U.append(mpi.COMM_WORLD.allgather(u))
    return X, U

In [160]:
%px xx, sol = update()

In [161]:
line_para = []
size = view['size'][0]
xx = view['xx'][0]
sol = view['sol'][0]

In [162]:
for i in range(size):
    line_para.append(Lines(x=xx[i], y=sol[0][i], scales={'x': x_sc, 'y': y_sc}))

In [163]:
fig_para = Figure(axes=[ax_x, ax_y], marks=line_para, animation_duration=100)
fig_para

Figure(animation_duration=100, axes=[Axis(label='x', scale=LinearScale(), side='bottom'), Axis(label='solution', orientation='vertical', scale=LinearScale(), side='left')], fig_margin={'top': 60, 'bottom': 60, 'left': 60, 'right': 60}, layout=Layout(min_width='125px'), marks=[Lines(colors=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf'], interactions={'hover': 'tooltip'}, scales={'x': LinearScale(), 'y': LinearScale()}, scales_metadata={'x': {'orientation': 'horizontal', 'dimension': 'x'}, 'y': {'orientation': 'vertical', 'dimension': 'y'}, 'color': {'dimension': 'color'}}, tooltip_style={'opacity': 0.9}, x=array([ 0.        ,  0.01010101,  0.02020202,  0.03030303,  0.04040404,
        0.05050505,  0.06060606,  0.07070707,  0.08080808,  0.09090909,
        0.1010101 ,  0.11111111,  0.12121212,  0.13131313,  0.14141414,
        0.15151515,  0.16161616,  0.17171717,  0.18181818,  0.19191919,
        0.2020202 ,  0.21212121,  0.22222222,  0.23232323,  0.24242424,
        0.25252525,  0.26262626,  0.27272727,  0.28282828]), y=array([ 0.        ,  0.00137741,  0.00265279,  0.00382614,  0.00489746,
        0.00586675,  0.00673401,  0.00749923,  0.00816243,  0.0087236 ,
        0.00918274,  0.00953984,  0.00979492,  0.00994796,  0.00999898,
        0.00994796,  0.00979492,  0.00953984,  0.00918274,  0.0087236 ,
        0.00816243,  0.00749923,  0.00673401,  0.00586675,  0.00489746,
        0.00382614,  0.00265279,  0.00137741,  0.        ])), Lines(colors=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf'], interactions={'hover': 'tooltip'}, scales={'x': LinearScale(), 'y': LinearScale()}, scales_metadata={'x': {'orientation': 'horizontal', 'dimension': 'x'}, 'y': {'orientation': 'vertical', 'dimension': 'y'}, 'color': {'dimension': 'color'}}, tooltip_style={'opacity': 0.9}, x=array([ 0.2020202 ,  0.21212121,  0.22222222,  0.23232323,  0.24242424,
        0.25252525,  0.26262626,  0.27272727,  0.28282828,  0.29292929,
        0.3030303 ,  0.31313131,  0.32323232,  0.33333333,  0.34343434,
        0.35353535,  0.36363636,  0.37373737,  0.38383838,  0.39393939,
        0.4040404 ,  0.41414141,  0.42424242,  0.43434343,  0.44444444,
        0.45454545,  0.46464646,  0.47474747,  0.48484848,  0.49494949,
        0.50505051,  0.51515152,  0.52525253]), y=array([ 0.        ,  0.00158147,  0.00306091,  0.00443832,  0.0057137 ,
        0.00688705,  0.00795837,  0.00892766,  0.00979492,  0.01056015,
        0.01122334,  0.01178451,  0.01224365,  0.01260076,  0.01285583,
        0.01300888,  0.01305989,  0.01300888,  0.01285583,  0.01260076,
        0.01224365,  0.01178451,  0.01122334,  0.01056015,  0.00979492,
        0.00892766,  0.00795837,  0.00688705,  0.0057137 ,  0.00443832,
        0.00306091,  0.00158147,  0.        ])), Lines(colors=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf'], interactions={'hover': 'tooltip'}, scales={'x': LinearScale(), 'y': LinearScale()}, scales_metadata={'x': {'orientation': 'horizontal', 'dimension': 'x'}, 'y': {'orientation': 'vertical', 'dimension': 'y'}, 'color': {'dimension': 'color'}}, tooltip_style={'opacity': 0.9}, x=array([ 0.44444444,  0.45454545,  0.46464646,  0.47474747,  0.48484848,
        0.49494949,  0.50505051,  0.51515152,  0.52525253,  0.53535354,
        0.54545455,  0.55555556,  0.56565657,  0.57575758,  0.58585859,
        0.5959596 ,  0.60606061,  0.61616162,  0.62626263,  0.63636364,
        0.64646465,  0.65656566,  0.66666667,  0.67676768,  0.68686869,
        0.6969697 ,  0.70707071,  0.71717172,  0.72727273,  0.73737374,
        0.74747475,  0.75757576,  0.76767677]), y=array([ 0.        ,  0.00158147,  0.00306091,  0.00443832,  0.0057137 ,
        0.00688705,  0.00795837,  0.00892766,  0.00979492,  0.01056015,
        0.01122334,  0.01178451,  0.01224365,  0.01260076,  0.01285583,
        0.0130088

In [164]:
import time
for k in range(100):
    fig_para.title = f"iteration {k}"
    for i in range(size):
        line_para[i].y=sol[k][i]
    time.sleep(.1)

In [169]:
sol = view.apply(update)

In [166]:
def update(nbite=100):
    U = []
    U.append(mpi.COMM_WORLD.allgather(u))
    X = mpi.COMM_WORLD.allgather(xl)
    for k in range(nbite):
        if rank == 0:
            mpi.COMM_WORLD.send(u[-1-2*overlap], rank + 1, rank)
            b[-1] = mpi.COMM_WORLD.recv(None, rank + 1, rank + 1)
        elif rank == size - 1:
            mpi.COMM_WORLD.send(u[2*overlap], rank - 1, rank)
            b[0] = mpi.COMM_WORLD.recv(None, rank - 1, rank - 1)
        else:
            mpi.COMM_WORLD.send(u[-1-2*overlap], rank + 1, rank)
            b[-1] = mpi.COMM_WORLD.recv(None, rank + 1, rank + 1)
            mpi.COMM_WORLD.send(u[2*overlap], rank - 1, rank)
            b[0] = mpi.COMM_WORLD.recv(None, rank - 1, rank - 1)
        u[:] = LU(b)
        U.append(mpi.COMM_WORLD.allgather(u))
    return X, U

In [174]:
sol[0][1]

[[array([ 0.        ,  0.00499949,  0.00989695,  0.01469238,  0.01938578,
          0.02397715,  0.02846648,  0.03285379,  0.03713907,  0.04132231,
          0.04540353,  0.04938272,  0.05325987,  0.057035  ,  0.06070809,
          0.06427916,  0.06774819,  0.07111519,  0.07438017,  0.07754311,
          0.08060402,  0.0835629 ,  0.08641975,  0.08917457,  0.09182736,
          0.09437812,  0.09682685,  0.09917355,  0.10141822]),
  array([ 0.08060402,  0.0835629 ,  0.08641975,  0.08917457,  0.09182736,
          0.09437812,  0.09682685,  0.09917355,  0.10141822,  0.10356086,
          0.10560147,  0.10754005,  0.10937659,  0.11111111,  0.1127436 ,
          0.11427405,  0.11570248,  0.11702887,  0.11825324,  0.11937557,
          0.12039588,  0.12131415,  0.12213039,  0.12284461,  0.12345679,
          0.12396694,  0.12437506,  0.12468115,  0.12488522,  0.12498725,
          0.12498725,  0.12488522,  0.12468115]),
  array([ 0.12345679,  0.12396694,  0.12437506,  0.12468115,  0.12488522,